In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from haversine import haversine
%matplotlib inline

# Module relatif à Gurobi
from gurobipy import *

# Module csv
import csv

In [2]:
path = "V1\instances\InstanceBordeauxV1.xlsx"
# Employees
df_Workers = pd.read_excel(path, sheet_name=0, index_col='EmployeeName')
# Employees unvabilites
df_Workers_un = pd.read_excel(path, sheet_name=1, index_col='EmployeeName')
# Task
df_Task = pd.read_excel(path, sheet_name=2, index_col='TaskId')
# Task unvabilites
df_Task_un = pd.read_excel(path, sheet_name=3, index_col='TaskId')

In [3]:
# Employees
dict_Workers = df_Workers.to_dict('index')
# Employees unvabilites
dict_Workers_un = df_Workers_un.to_dict('index')
# Task
dict_Task = df_Task.to_dict('index')
# Task unvabilites
dict_Task_un = df_Task_un.to_dict('index')

In [4]:
dict_Workers

{'Valentin': {'Latitude': 45.15121765523164,
  'Longitude': -0.822092647754919,
  'Skill': 'Oenology',
  'Level': 2,
  'WorkingStartTime': '8:00am',
  'WorkingEndTime': '6:00pm'},
 'Ambre': {'Latitude': 45.15121765523164,
  'Longitude': -0.822092647754919,
  'Skill': 'Oenology',
  'Level': 1,
  'WorkingStartTime': '8:00am',
  'WorkingEndTime': '6:00pm'}}

In [5]:
dict_Task

{'T1': {'Latitude': 44.55654938342008,
  'Longitude': -0.3193922422375719,
  'TaskDuration': 60,
  'Skill': 'Oenology',
  'Level': 1,
  'OpeningTime': '8:00am',
  'ClosingTime': '6:00pm'},
 'T2': {'Latitude': 44.96750095217799,
  'Longitude': -0.6086852638150881,
  'TaskDuration': 60,
  'Skill': 'Oenology',
  'Level': 1,
  'OpeningTime': '8:00am',
  'ClosingTime': '6:00pm'},
 'T3': {'Latitude': 45.14421541464031,
  'Longitude': -0.7342570469020379,
  'TaskDuration': 60,
  'Skill': 'Oenology',
  'Level': 2,
  'OpeningTime': '8:00am',
  'ClosingTime': '6:00pm'},
 'T4': {'Latitude': 45.264808304867096,
  'Longitude': -0.7717887212411139,
  'TaskDuration': 60,
  'Skill': 'Oenology',
  'Level': 1,
  'OpeningTime': '8:00am',
  'ClosingTime': '6:00pm'},
 'T5': {'Latitude': 45.044422793402624,
  'Longitude': -0.6687606009488057,
  'TaskDuration': 60,
  'Skill': 'Oenology',
  'Level': 2,
  'OpeningTime': '8:00am',
  'ClosingTime': '6:00pm'},
 'T6': {'Latitude': 45.19957452440505,
  'Longitude':

#### Sets

In [6]:
Workers = list(df_Workers.index)
Skills = list(df_Workers["Skill"].unique())
Tasks = list(df_Task.index)

#### Parameters

level_skill: level of worker (in Workers) in some skill (of Skills)

In [7]:
def time_to_minutes(time_str):
    str_hour = time_str[:-2]
    am_pm = time_str[-2:]
    hour_str, minute = str_hour.split(':')
    hour = int(hour_str)
    if am_pm == 'pm':
        hour += 12
    return hour * 60 + int(minute)

In [8]:
df_aux = pd.DataFrame()
for skill in Skills:
    df_aux[skill] = df_Workers.apply(lambda x: x['Level'] if x['Skill'] == skill else 0, axis=1)
l = df_aux.to_dict('index')
l

{'Valentin': {'Oenology': 2}, 'Ambre': {'Oenology': 1}}

In [9]:
# Opening time for taks i
a = df_Task.apply(lambda x: time_to_minutes(x['OpeningTime']), axis=1).to_dict()
# Closing time for taks i
b = df_Task.apply(lambda x: time_to_minutes(x['ClosingTime']), axis=1).to_dict()

# time worker w start working
alpha = df_Workers.apply(lambda x: time_to_minutes(x['WorkingStartTime']), axis=1).to_dict()
# time worker w end working
beta = df_Workers.apply(lambda x: time_to_minutes(x['WorkingEndTime']), axis=1).to_dict()

# Duration of the task i
d = df_Task['TaskDuration'].to_dict()
# Skill requierd by task i
s = df_Task['Skill'].to_dict()

{'T1': 'Oenology',
 'T2': 'Oenology',
 'T3': 'Oenology',
 'T4': 'Oenology',
 'T5': 'Oenology',
 'T6': 'Oenology',
 'T7': 'Oenology',
 'T8': 'Oenology',
 'T9': 'Oenology',
 'T10': 'Oenology'}

In [10]:
# Level requierd by task i on the skill s
df_aux = pd.DataFrame()
for skill in Skills:
    df_aux[skill] = df_Task.apply(lambda x: x['Level'] if x['Skill'] == skill else 100, axis=1)
r = df_aux.to_dict('index')
r

{'T1': {'Oenology': 1},
 'T2': {'Oenology': 1},
 'T3': {'Oenology': 2},
 'T4': {'Oenology': 1},
 'T5': {'Oenology': 2},
 'T6': {'Oenology': 2},
 'T7': {'Oenology': 1},
 'T8': {'Oenology': 1},
 'T9': {'Oenology': 2},
 'T10': {'Oenology': 1}}

In [11]:
def read_lat_log(df):
    nodes = []
    for name, dic_inf in df.items():
        nodes.append([name,(dic_inf["Latitude"], dic_inf["Longitude"])])
    return nodes

def haversine(pt1, pt2):
    R = 6371  # radius of the Earth in kilometers
    lat1, lon1, lat2, lon2 = map(np.radians, [pt1[0], pt1[1], pt2[0], pt2[1]])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

def distance_matrix(nodes):
    # Create distance matrix with haversine distance
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i, n):
            dist = haversine(nodes[i], nodes[j])
            dist_matrix[i][j] = dist
            dist_matrix[j][i] = dist
    return dist_matrix


In [12]:
def read_lat_log(df):
    nodes = {}
    for name, dic_inf in df.items():
        nodes[name] = (dic_inf["Latitude"], dic_inf["Longitude"])
    return nodes

def haversine(pt1, pt2):
    R = 6371  # radius of the Earth in kilometers
    lat1, lon1, lat2, lon2 = map(np.radians, [pt1[0], pt1[1], pt2[0], pt2[1]])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

def distance_matrix(nodes):
    # Create distance matrix with haversine distance
    dist_matrix = {}
    for node_i in nodes.keys():
        dist_matrix[node_i] = {}
        for node_j in nodes.keys():
            dist = haversine(nodes[node_i], nodes[node_j])
            dist_matrix[node_i][node_j] = np.ceil(dist / 50 * 60)
    return dist_matrix


In [13]:
# Denfine the nodes
nodes = read_lat_log(dict_Workers)| read_lat_log(dict_Task)

## Define the time matrix in minutes round ceil between a node (task or worker) with another (task or worker)
t = distance_matrix(nodes)
t['Valentin']['T1']

93.0

### Les variables

In [14]:
def init_param():
    return Workers, Skills, Tasks, l, a, b, alpha, beta, d, s, r, t

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-26
